In [1]:
##  Adapted from https://www.geeksforgeeks.org/bloom-filters-introduction-and-python-implementation/ 

import math 
import mmh3 
from bitarray import bitarray 

class BloomFilter(object): 

	''' 
	Class for Bloom filter, using murmur3 hash function 
	'''

	def __init__(self, items_count,fp_prob): 
		''' 
		items_count : int 
			Number of items expected to be stored in bloom filter 
		fp_prob : float 
			False Positive probability in decimal 
		'''
		# False posible probability in decimal 
		self.fp_prob = fp_prob 

		# Size of bit array to use 
		self.size = self.get_size(items_count,fp_prob) 

		# number of hash functions to use 
		self.hash_count = self.get_hash_count(self.size,items_count) 

		# Bit array of given size 
		self.bit_array = bitarray(self.size) 

		# initialize all bits as 0 
		self.bit_array.setall(0) 

	def add(self, item): 
		''' 
		Add an item in the filter 
		'''
		digests = [] 
		for i in range(self.hash_count): 

			# create digest for given item. 
			# i work as seed to mmh3.hash() function 
			# With different seed, digest created is different 
			digest = mmh3.hash(item,i) % self.size 
			digests.append(digest) 

			# set the bit True in bit_array 
			self.bit_array[digest] = True

	def check(self, item): 
		''' 
		Check for existence of an item in filter 
		'''
		for i in range(self.hash_count): 
			digest = mmh3.hash(item,i) % self.size 
			if self.bit_array[digest] == False: 

				# if any of bit is False then,its not present 
				# in filter 
				# else there is probability that it exist 
				return False
		return True

	@classmethod
	def get_size(self,n,p): 
		''' 
		Return the size of bit array(m) to used using 
		following formula 
		m = -(n * lg(p)) / (lg(2)^2) 
		n : int 
			number of items expected to be stored in filter 
		p : float 
			False Positive probability in decimal 
		'''
		m = -(n * math.log(p))/(math.log(2)**2) 
		return int(m) 

	@classmethod
	def get_hash_count(self, m, n): 
		''' 
		Return the hash function(k) to be used using 
		following formula 
		k = (m/n) * lg(2) 

		m : int 
			size of bit array 
		n : int 
			number of items expected to be stored in filter 
		'''
		k = (m/n) * math.log(2) 
		return int(k) 


### Testing the bloom filter implementation

In [11]:
%time
import json
import datetime
import timeit

def generate_random_string(N):
    return ''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(N))

def simple_test():
    bf = BloomFilter(1000, .001)
    for i in range(1000):
        bf.add(i)
        if not bf.check(i):
            print("False Negative!")

    count = 0.0
    fp = 0.0
    for i in range(1001, 10000):
        if bf.check(i):
            fp+=1
        count += 1
    print("Result of simple test:")
    print("False Positive Rate: " + str(fp / count))

def string_test():
    bf = BloomFilter(1000, .001)
    for i in range(1000):
        random_string = generate_random_string(i)
        bf.add(random_string)
        assert(bf.check(random_string))

    count = 0.0
    fp = 0.0
    for i in range(1001, 10000):
        random_string = generate_random_string(i)
        if bf.check(random_string):
            fp+=1
        count += 1
    print("Resuls of string test:")
    print("False Positive Rate: " + str(fp / count))
    

def url_test(positives, negatives, fp_rate):
    bloom_capacity = len(positives)
    bf = BloomFilter(bloom_capacity, fp_rate)
    count = 0
    
    start = timeit.default_timer()
    before = datetime.datetime.now()
    for pos in positives:
        if count == bloom_capacity:
            break
        bf.add(pos)
        count += 1
        assert(bf.check(pos))
    
    after = datetime.datetime.now()
    stop = timeit.default_timer()
    delta = after - before
    print("Build time:", delta)
    print("Timeit time", (stop-start))
    print("Bits needed", bf.size)
    print("Converting bits to bytes", (bf.size)/8)
    print("Hash functions needed", bf.hash_count)

    fp = 0.0
    for neg in negatives:
        if bf.check(neg):
            fp += 1
    print("Results of url test:")
    print("False positives", fp / len(negatives))
    return bf

with open('../data/dataset.json', 'r') as f:
    dataset = json.load(f)

positives = dataset['positives']
negatives = dataset['negatives']
data_fraction = 1
positives = positives[:int(data_fraction * len(positives))]
negatives = negatives[:int(data_fraction * len(negatives))]

print("Total Dataset: ", len(positives) + len(negatives))
print("Number of positive instances: ", len(positives))
print("Number of negative instances: ", len(negatives))

filter = url_test(positives, negatives, 0.01)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.72 µs
Total Dataset:  2926705
Number of positive instances:  1491178
Number of negative instances:  1435527
Build time: 0:00:10.957578
Timeit time 10.957599513232708
Bits needed 14293028
Converting bits to bytes 1786628.5
Hash functions needed 6
Results of url test:
False positives 0.010326521200924817


In [13]:
start = timeit.default_timer()
predict = filter.check(positives[0])
end = timeit.default_timer()
print("Prediction time", (end - start))

Prediction time 9.690038859844208e-05


#### Results obtained on Half Dataset

CPU times: user 3 µs, sys: 0 ns, total: 3 µs  
Wall time: 5.48 µs  
Total Dataset:  1463352  
Number of positive instances:  745589  
Number of negative instances:  717763  
Build time: 0:00:05.510930  
Bits needed 7146514  
Converting bits to bytes 893314.25  
Hash functions needed 6  
Results of url test:  
False positives 0.010298664043702447  
Size in Mb: 0.89331425  

#### Results obtained on full dataset

CPU times: user 3 µs, sys: 0 ns, total: 3 µs  
Wall time: 5.72 µs  
Total Dataset:  2926705  
Number of positive instances:  1491178  
Number of negative instances:  1435527  
Build time: 0:00:10.896063  
Bits needed 14293028  
Converting bits to bytes 1786628.5  
Hash functions needed 6  
Results of url test:  
False positives 0.010326521200924817  
Size in Mb: 1.7866285  